In [1]:
# -*- coding: utf-8 -*-
import io
import pprint
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from oauth2client.service_account import ServiceAccountCredentials
from httplib2 import Http

In [2]:
SERVICE_ACC_KEY_FILE_NAME = "service_acc_key.json"

# SCOPES에 들어가는 값들은 다음 URL을 참조: https://developers.google.com/drive/api/v3/about-auth
SCOPES = [
    "https://www.googleapis.com/auth/drive.readonly"
]

credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACC_KEY_FILE_NAME, SCOPES)
http_auth = credentials.authorize(Http())
drive = build("drive", "v3", http=http_auth)

## 특정 파일/폴더의 MetaData를 출력

In [3]:
FILE_ID = "1EFzqg-bfKXOjHP3q7NNUM6QYuEln8sR5"  # 샘플로 읽어볼 파일의 ID
FOLDER_ID = "1eZDWJyWm_ORYzldgrEtcZTGIINfrrXdk"  # 샘플로 읽어볼 폴더의 ID

# fields에 들어가는 인자는 다음 URL을 참조: https://developers.google.com/drive/api/v3/reference/files
FIELDS = "kind, id, name, mimeType, size"

print("\n##### File's MetaData #####\n")
response = drive.files().get(fileId=FILE_ID, fields=FIELDS).execute()
pprint.pprint(response)

print("\n\n##### Folder's MetaData #####\n")
response = drive.files().get(fileId=FOLDER_ID, fields=FIELDS).execute()
pprint.pprint(response)


##### File's MetaData #####

{'id': '1EFzqg-bfKXOjHP3q7NNUM6QYuEln8sR5',
 'kind': 'drive#file',
 'mimeType': 'text/plain',
 'name': 'Test.txt',
 'size': '10'}


##### Folder's MetaData #####

{'id': '1eZDWJyWm_ORYzldgrEtcZTGIINfrrXdk',
 'kind': 'drive#file',
 'mimeType': 'application/vnd.google-apps.folder',
 'name': 'Test'}


## 특정 파일을 다운로드 받아서 출력

In [4]:
print("\n##### Download Start #####\n")
request = drive.files().get_media(fileId=FILE_ID)
buffer = io.BytesIO()
downloader = MediaIoBaseDownload(buffer, request)
done = False

while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

print("\n\n##### Download Result #####\n")
print(buffer.getvalue())


##### Download Start #####

Download 100%.


##### Download Result #####

b'Test Body~'


## 특정 폴더내의 폴더/파일들 출력

In [5]:
response = drive.files().list(
    q="'{0}' in parents".format(FOLDER_ID),
    spaces="drive",
    fields="files(id, name, mimeType, parents)"
).execute()

files = response.get("files", [])

print("\n##### 폴더/파일 목록 #####")
for f in files:
    print("")
    pprint.pprint(f)


##### 폴더/파일 목록 #####

{'id': '1Lde5wC3wacAmUIIiVZ4qDFWd89IouOft',
 'mimeType': 'application/vnd.google-apps.folder',
 'name': 'sub_folder2',
 'parents': ['1eZDWJyWm_ORYzldgrEtcZTGIINfrrXdk']}

{'id': '17lrAKEA73zf_EAuimoMmyebVAg-8p2GZ',
 'mimeType': 'application/vnd.google-apps.folder',
 'name': 'sub_folder1',
 'parents': ['1eZDWJyWm_ORYzldgrEtcZTGIINfrrXdk']}

{'id': '1T0Qb5QsHh9C3lAgDzCtHWhPTZtoD98X2',
 'mimeType': 'text/plain',
 'name': 'test2.txt',
 'parents': ['1eZDWJyWm_ORYzldgrEtcZTGIINfrrXdk']}

{'id': '1Gh1Xdgp9FLFfsf_20Ou_V_mULhz9ZXk5',
 'mimeType': 'text/plain',
 'name': 'test1.txt',
 'parents': ['1eZDWJyWm_ORYzldgrEtcZTGIINfrrXdk']}

{'id': '1EFzqg-bfKXOjHP3q7NNUM6QYuEln8sR5',
 'mimeType': 'text/plain',
 'name': 'Test.txt',
 'parents': ['1eZDWJyWm_ORYzldgrEtcZTGIINfrrXdk']}
